In [74]:
import pandas as pd
import numpy as np
import plotly.express as px
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## Conexion a la BD

In [10]:
def dataBase_connection():
    """This function connects with the data Warehouse and return a diccionary with the dataframes"""

    host = '127.0.0.1'  # Cambia esto por el endpoint de tu RDS
    port = 3307
    user = 'airflow'  # Cambia esto por tu usuario
    password = 'airflow'  # Cambia esto por tu contraseña
    database = 'airflow'  # Cambia esto por tu base de datos
    engine = None

    
    engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}:{port}/{database}')
    

    # Create a list of the tables in the database
    tables =['temperature_WH','humidity_WH','wind_speed_WH','pressure_WH','weather_description_WH','wind_direction_WH']
   
    dataframes = {}
    # Loop through the tables and read them into pandas dataframes
    for table in tables:
        df = pd.read_sql_table(table, engine)
        dataframes[table] = df

    # Close the connection
    engine.dispose()
    return dataframes

In [ ]:
#Get the dataframes from the database
dataframes = dataBase_connection()

In [ ]:
#List of the names of the tables in the database
dataframes.keys()

dict_keys(['temperature_WH', 'humidity_WH', 'wind_speed_WH', 'pressure_WH', 'weather_description_WH', 'wind_direction_WH'])

## Temperature

In [ ]:

# Change the type of data int the columns datetime 
dataframes['temperature_WH']['datetime'] = pd.to_datetime(dataframes['temperature_WH']['datetime'])


In [ ]:
dataframes['temperature_WH']['datetime'].dt.year.unique()

In [ ]:
def graphic(df,year,city):
    """This function creates a graphic with the temperature data for a specific year"""

    #Filter the dataframe to get only the data of 2023
    temperature_df= df[df['datetime'].dt.year == year].copy()

    #Get the average of the temperature by month
    temperature_df['month'] = temperature_df['datetime'].dt.month
    
    #Change the name of the month
    temperature_df['Month'] = temperature_df['month'].replace({1: 'January', 2: 'February', 3: 'March', 4: 'April', 5: 'May', 6: 'June', 7: 'July', 8: 'August', 9: 'September', 10: 'October', 11: 'November', 12: 'December'})


    #Get the average of the temperature by month
    temperature_df = temperature_df.groupby(['Month']).mean().reset_index()

    # Show int ascending order
    temperature_df = temperature_df.sort_values(by='month', ascending=True)

    # Graphic of the temperature where the x axis are the months of a year and the y axis are the temperature in Farenheit degrees
    fig = go.Bar( x = temperature_df['Month'], y = temperature_df[city], name = f'Temperature {year}', 
            marker= dict(color=temperature_df[city], colorscale = [[0,'rgb(255,200,200)'],[1,'rgb(255,0,0)']],# scale from red to white
                            colorbar=dict(title='Temp °F')))
   

    return fig



In [ ]:
def linear_graphic(df,city):
    """This function creat a linear grophic where show the temperature avarage for each year in a specific city"""
    #Get the average of the temperature by year
    df['year'] = df['datetime'].dt.year

    #Get the average of the temperature by year
    df = df.groupby(['year']).mean().reset_index()

    # Graphic of the temperature where the x axis are the months of a year and the y axis are the temperature in Farenheit degrees
    fig = go.Figure(data=go.Scatter(x=df['year'], y=df[city], mode='lines+markers', name=f'Temperature {city}'))
    
    return fig


### Los Angeles

#### Graphics where show the temperature by month from 2012 to 2017

In [ ]:
# Graphic of the temperature where the x axis are the months of a year and the y axis are the temperature in Farenheit degrees
fig = make_subplots(rows=3, cols=2, subplot_titles=('2012', '2013', '2014', '2015', '2016', '2017'))
fig.add_trace(graphic(dataframes['temperature_WH'],2012,'Los Angeles'), row=1, col=1)
fig.add_trace(graphic(dataframes['temperature_WH'],2013,'Los Angeles'), row=1, col=2)
fig.add_trace(graphic(dataframes['temperature_WH'],2014,'Los Angeles'), row=2, col=1)
fig.add_trace(graphic(dataframes['temperature_WH'],2015,'Los Angeles'), row=2, col=2)
fig.add_trace(graphic(dataframes['temperature_WH'],2016,'Los Angeles'), row=3, col=1)
fig.add_trace(graphic(dataframes['temperature_WH'],2017,'Los Angeles'), row=3, col=2)

fig.update_layout(
    height=800, width=1000,
    title_text="Temperatures in Los Angeles from 2012 to 2017",
    showlegend=False
)

fig.show()


#### Graphic where show the average temperature from 2012 to 2017

In [85]:
#Get the rsult of the function linear_graphic
fig = linear_graphic(dataframes['temperature_WH'],'Los Angeles')
fig.update_layout(
    title="Temperature in Los Angeles from 2012 to 2017",
    xaxis_title="Year",
    yaxis_title="Temperature °F",
    showlegend=True
)
fig.show()

#### Graphic where show the growth rate by year

In [88]:
def growth_rate(df,city):
    """This function calculates  an graphs the growth rate of the temperature in a specific city"""
    #Get the average of the temperature by year
    df['year'] = df['datetime'].dt.year

    #Get the average of the temperature by year
    df = df.groupby(['year']).mean().reset_index()

    #Calculate the growth rate
    df['growth_rate'] = df[city].pct_change() * 100

    # Graphic of the temperature where the x axis are the months of a year and the y axis are the temperature in Farenheit degrees
    fig = go.Figure(data=go.Scatter(x=df['year'], y=df['growth_rate'], mode='lines+markers', name=f'Growth rate {city}'))
    

    return fig

In [90]:
fig = growth_rate(dataframes['temperature_WH'],'Los Angeles')
fig.update_layout(
    title="Growth rate of temperature in Los Angeles from 2012 to 2017",
    xaxis_title="Year",
    yaxis_title="Growth rate %",
    showlegend=True
)
# 📋 Agregamos línea cero para referencia visual
fig.add_hline(y=0, line_dash="dot", line_color="gray")
fig.show()

fig = 